In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://docs.quantum.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

qiskit_runtime_service.__init__:INFO:2024-04-29 13:15:35,275: Default instance: ibm-q/open/main


In [54]:
#installing the necessary packages

!pip install qiskit_machine_learning
!pip install qiskit_optimization

In [43]:
#importing libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from qiskit_aer import AerSimulator
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import VQC
from qiskit_algorithms.optimizers import COBYLA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [44]:
# Set random seed for reproducibility

np.random.seed(60)

In [45]:
#loading the iris dataset

iris = datasets.load_iris()
features = iris.data
labels = iris.target

In [46]:
# Printing the first 5 rows of the features of the dataset

print("First 5 rows of the features of the dataset:\n")
print(features[:5])

First 5 rows of the features of the dataset:

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [47]:
# Printing the first 5 labels

print("First 5 labels of the dataset:\n")
print(labels[:5])

First 5 labels of the dataset:

[0 0 0 0 0]


In [6]:
#preprocessing the data

features = MinMaxScaler(feature_range=(0, 1)).fit_transform(features)

print("Features after applying MinMaxScaler:\n")
print(features[:5])

Features after applying MinMaxScaler:

[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]


In [48]:
# Splitting the dataset into training and test sets

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, train_size=0.8, random_state=30)

In [49]:
# Set up quantum feature map
num_qubits = features.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement="linear")
print(feature_map)

     ┌────────────────────────────────────┐
q_0: ┤0                                   ├
     │                                    │
q_1: ┤1                                   ├
     │  ZZFeatureMap(x[0],x[1],x[2],x[3]) │
q_2: ┤2                                   ├
     │                                    │
q_3: ┤3                                   ├
     └────────────────────────────────────┘


In [50]:
# Set up VQC model with COBYLA optimizer
optimizer = COBYLA(maxiter=100)
vqc = VQC(optimizer=optimizer, feature_map=feature_map)

# Train the VQC model
vqc.fit(train_features, train_labels)


In [52]:
# Predicting labels for test_features

pred_labels = vqc.predict(test_features)